In [2]:
import pandas as pd
from sklearn import preprocessing

In [3]:
school_data = pd.read_csv('buildbps.csv', encoding='latin1')
school_data.head()



,SMMA_Identifier,SMMA_Only_For_Map,BPS_School_Name,BPS_Historical_Name,SMMA_Abbreviated_Name,BPS_Address,BRA_Neighborhood,SMMA_latitude,SMMA_longitude,SMMA_Typology,...,SMMA_EA_K8_Adequacy_Cafeteria,SMMA_EA_K8_Adequacy_Stage,SMMA_EA_K8_Adequacy_Medical,SMMA_EA_K8_Adequacy_Administration,SMMA_EA_K8_Adequacy_Custodial,SMMA_EA_K8_Adequacy_Network,SMMA_EA_K8_Adequacy_Other_1,SMMA_EA_K8_Adequacy_Other_2,SMMA_EA_K8_Adequacy_Other_3,SMMA_EA_K8_Overall_EFE_spaces
0,031,NaN,"Adams, Samuel Elementary",Adams,Adams,"165 Webster St East Boston, MA 02128",East Boston,42.365553,-71.034917,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,078,NaN,"Alighieri, Dante Montessori School",Alighieri,Alighieri,"37 Gove Street East Boston, MA 02128",East Boston,42.371565,-71.037608,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,045A,NaN,Another Course to College*,Taft,ACC*,"20 Warren Street Brighton, MA 02135",Allston,42.350354,-71.145582,High School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,012,NaN,Baldwin Early Learning Pilot Academy,Baldwin ELC,Baldwin,"121 Corey Rd Brighton, MA 02135",Brighton,42.342037,-71.140529,Early Learning,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,087,NaN,"Bates, Phineas Elementary",Bates,Bates,"426 Beech St Roslindale, MA 02131",Roslindale,42.277663,-71.135353,Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# def replaceFunction(row):
#     outputrow = row
#     map_dictionary = {"Not Present":1, 
#                       "Replace":2,
#                       "Moderate":3,
#                       "Minor":4,
#                       "Adequate":5,
#                       "Deficient":1,
#                       "Poor":2,
#                       "Fair":3,
#                       "Good":4,
#                       "Excellent":5
#                      }
#     for key in map_dictionary:
#         outputrow = outputrow.replace(to_replace= key, value = map_dictionary[key])
#     return outputrow

# new_data = school_data.apply(replaceFunction)
def schoolEvaluation(row):
    total = 0
    score = 0
    value_counts = row.value_counts()
    map_dictionary = {"Not Present":1, 
                      "Replace":2,
                      "Moderate":3,
                      "Minor":4,
                      "Adequate":5,
                      "Deficient":1,
                      "Poor":2,
                      "Fair":3,
                      "Good":4,
                      "Excellent":5
                     }
    map_dictionary_test = {"Not Present":0, 
                      "Replace":1,
                      "Moderate":2,
                      "Minor":3,
                      "Adequate":4,
                      "Deficient":0,
                      "Poor":1,
                      "Fair":2,
                      "Good":3,
                      "Excellent":4
                     }
    for key in map_dictionary_test:
        if (key in value_counts):
            score += value_counts[key] * map_dictionary_test[key]
            total += value_counts[key] * 5
    if total != 0:
        return (row.at["BPS_School_Name"], score/total)
    return (row.at["BPS_School_Name"], 0)
new_data = school_data.apply(schoolEvaluation, axis = 1)

highest_value = 0
best_school = "school"
for school in new_data:
    if school[1] > highest_value:
        highest_value = school[1]
        best_school = school[0]

print(highest_value)
print(best_school)

0.7016393442622951
Frederick, Lilla G. Middle
